In [1]:
import sys
sys.path
sys.path.append(r"C:\Users\yanpe\.env\Lib\site-packages")
print(sys.path)

['C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\\python312.zip', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\\DLLs', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\\Lib', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0', '', 'C:\\Users\\yanpe\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages', 'C:\\Users\\yanpe\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\win32', 'C:\\Users\\yanpe\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\win32\\lib', 'C:\\Users\\yanpe\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_

In [1]:
import os
import open3d as o3d
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import torch_geometric.transforms as T
from torch_geometric.typing import WITH_TORCH_CLUSTER
from pyg_pointnet2 import PyGPointNet2, PyGPointNet2NoColor
from pc_label_map import color_map

if not WITH_TORCH_CLUSTER:
    quit("This example requires 'torch-cluster'")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Empty the CUDA cache
torch.cuda.empty_cache()

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#model = PyGPointNet2(num_classes=13).to(device) # x6 model
model = PyGPointNet2NoColor(num_classes=13).to(device) # x3 model


In [4]:
#model_file_path = "checkpoints/pointnet2_s3dis_pretransform_seg_x6_45_checkpoint.pth" # Orignial pretransformed x6 model
#model_file_path = "checkpoints/pointnet2_s3dis_transform_seg_x6_45_checkpoint.pth" # Orignial transformed x6 model
#model_file_path = "checkpoints/pointnet2_s3dis_pretransform_seg_x3_45_checkpoint.pth" # Orignial pretransformed x3 model
#model_file_path = "checkpoints/pointnet2_s3dis_transform_seg_x3_45_checkpoint.pth" # Orignial transformed x3 model
model_file_path = "checkpoints/pointnet2_smartlab_sim_transform_seg_x3_100_checkpoint.pth" # Simulated transformed x3 model
#model_file_path = "checkpoints/pointnet2_smartlab_sim_pretransform_seg_x3_100_checkpoint.pth" # Simulated pretransformed x3 model
#model_file_path = "checkpoints/smartlab_fine_tuning_x3_50_20250416.pth" # Fine-tuned model


In [5]:
# Load the checkpoint dictionary
checkpoint = torch.load(model_file_path, map_location=device)
# Extract the model state dictionary
model_state_dict = checkpoint['model_state_dict']


In [6]:
model.load_state_dict(model_state_dict)
model.eval()

PyGPointNet2NoColor(
  (sa1_module): SAModule(
    (conv): PointNetConv(local_nn=MLP(6, 64, 64, 128), global_nn=None)
  )
  (sa2_module): SAModule(
    (conv): PointNetConv(local_nn=MLP(131, 128, 128, 256), global_nn=None)
  )
  (sa3_module): GlobalSAModule(
    (nn): MLP(259, 256, 512, 1024)
  )
  (fp3_module): FPModule(
    (nn): MLP(1280, 256, 256)
  )
  (fp2_module): FPModule(
    (nn): MLP(384, 256, 128)
  )
  (fp1_module): FPModule(
    (nn): MLP(131, 128, 128, 128)
  )
  (mlp): MLP(128, 128, 128, 13)
  (lin1): Linear(in_features=128, out_features=128, bias=True)
  (lin2): Linear(in_features=128, out_features=128, bias=True)
  (lin3): Linear(in_features=128, out_features=13, bias=True)
)

In [7]:
# Pcd files
#pcd_path = "C:/Users/yanpe/OneDrive - Metropolia Ammattikorkeakoulu Oy/Research/data/smartlab/Smartlab-2024-04-05_10-58-26_colour_cleaned.pcd"
pcd_path = "C:/Users/yanpe/OneDrive - Metropolia Ammattikorkeakoulu Oy/Research/data/smartlab/SmartLab_2024_E57_Single_5mm.pcd"
#pcd_path = "C:/Users/yanpe/OneDrive - Metropolia Ammattikorkeakoulu Oy/Research/data/smartlab/SmartLab_2024_E57_Single_5mm.ply"
pcd = o3d.io.read_point_cloud(pcd_path)

In [130]:
print(len(pcd.points))
print(len(pcd.colors))
print(len(pcd.normals))

35966772
35966772
0


In [8]:
# Move the point cloud to its min(x,y,z) corner
 
def move_to_corner(points):    
    # Find the minimum x, y, z
    min_xyz = points.min(axis=0)
    # Translate the point cloud so that the min corner becomes the origin
    moved_points = points - min_xyz
    
    return moved_points

moved_points = move_to_corner(np.array(pcd.points))
pcd.points = o3d.utility.Vector3dVector(moved_points)

In [109]:
# Check the range of points

def array_range(array):
    # Get min and max for each column
    min_values = np.min(array, axis=0)
    max_values = np.max(array, axis=0)

    # Display ranges for each column
    for i, (min_val, max_val) in enumerate(zip(min_values, max_values), start=1):
        print(f"Column {i}: min = {min_val}, max = {max_val}")

print(array_range(np.array(pcd.points)))

Column 1: min = 0.0, max = 10.791000008583069
Column 2: min = 0.0, max = 11.177000045776367
Column 3: min = 0.0, max = 3.4570000171661377
None


In [49]:
#if not pcd.has_normals():
#   pcd.estimate_normals()

In [9]:
# Downsample the point cloud with a voxel of 0.02
downpcd = pcd.voxel_down_sample(voxel_size=0.03)

print(len(downpcd.points))

866900


In [20]:
o3d.visualization.draw_geometries([downpcd], point_show_normal=False)

In [1]:
def normalize_points_center(points):
    # Step 1: Center the points around the origin
    centroid = np.mean(points, axis=0)
    centered_points = points - centroid

    # Step 2: Scale to fit within the [0, 1] interval
    min_vals = np.min(centered_points, axis=0)
    max_vals = np.max(centered_points, axis=0)
    scale = max_vals - min_vals

    # Avoid division by zero in case of flat dimension
    scale[scale == 0] = 1  # Set zero scales to 1 to keep that dimension as 0.5 after normalization

    normalized_points = (centered_points - min_vals) / scale

    return normalized_points

# normalized = normalize_points_center(np.array(downpcd.points))

In [10]:
def normalize_points_corner(points):
    # Step 1: Shift points so that the minimum x, y, z becomes the origin.
    min_vals = np.min(points, axis=0)
    shifted_points = points - min_vals  # Now the lower bound is (0,0,0)
    
    # Step 2: Compute the scaling factors from the shifted points.
    # The maximum after shifting represents the range in each dimension.
    max_vals = np.max(shifted_points, axis=0)
    scale = max_vals.copy()
    
    # Avoid division by zero if any dimension is flat.
    scale[scale == 0] = 1
    
    # Normalize the shifted points to the [0, 1] interval.
    normalized_points = shifted_points / scale

    return normalized_points

normalized = normalize_points_corner(np.array(downpcd.points))

In [11]:
# Extract coordinates and colors from the point cloud
down_points = torch.tensor(np.array(downpcd.points), dtype=torch.float32)  
down_colors = torch.tensor(np.array(downpcd.colors), dtype=torch.float32)
down_normalized = torch.tensor(normalized, dtype=torch.float32)

In [193]:
# 6 Features data

# Concatenate coordinates and colors to form the input features
x = torch.cat([down_colors, down_normalized], dim=1) # 6 (x)
# Create a Data object with x (6 features) and pos (coordinates)
data = Data(x=x, pos=down_points)

#pre_transform = T.NormalizeScale()
#data = pre_transform(data)


In [12]:
# 3 Features data

# Create a Data object with x (3 features) and pos (coordinates)
data = Data(x=down_normalized, pos=down_points)

#data = data.to(device)

#pre_transform = T.NormalizeScale()
#data = pre_transform(data)

In [88]:
import os
os.cpu_count()

22

In [14]:
# If you have only one point cloud
dataset = [data]  # List of Data objects

num_workers = 20
batch_size = 32

# Create a DataLoader (batch_size can be adjusted as needed)
custom_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False,
                         num_workers=num_workers, pin_memory=True)

In [15]:
import time

model.eval()

import torch.profiler

with torch.profiler.profile(
    activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA],
    record_shapes=True,
) as prof:

    with torch.no_grad():
        start_time = time.time()
        for data in custom_loader:
            data = data.to(device)
            with torch.amp.autocast("cuda"):
                predictions = model(data)
            labels = predictions.argmax(dim=-1)
            # Process the labels as needed
            #labels_arr = labels.cpu().numpy()
            # Count occurrences of labels
            unique_labels, label_counts = torch.unique(labels, return_counts=True)
            # Combine and print
            #result_labels = np.array(list(zip(unique_labels, label_counts)))
            #print("Label counts:")
            #for label, count in zip(unique_labels, label_counts):
            #    print(f"Label {label.item()}: {count.item()}")
            result_labels = torch.stack((unique_labels, label_counts), dim=1).cpu()
            print("Label counts:")
            print(result_labels)
        end_time = time.time()
        print(f"Total inference time: {end_time - start_time:.4f} seconds")  
    
print(prof.key_averages().table(sort_by="cuda_time_total"))



Label counts:
tensor([[     0, 170832],
        [     1, 125569],
        [     2, 360083],
        [     8,    901],
        [    12, 209515]])
Total inference time: 261.8178 seconds
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                     torch_cluster::fps         0.11%     291.020ms         0.15%     390.980ms     195.490ms      242.514s        94.01%      242.614s      121.307s             2  
enumerate(DataLoader)#_

In [21]:
# Save profiler output to csv
import csv

# Generate the profiling table
table_str = prof.key_averages().table(sort_by="cuda_time_total")

# Split into lines and clean
lines = table_str.split('\n')
header = lines[6].strip().split()
rows = [line.strip().split(None, len(header)-1) for line in lines[7:] if line.strip()]

# Save as CSV
profiler_record = "../docs/original_x6_prediction_profiler_output.csv"
with open(profiler_record, "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(rows)

batch_size 32, num_workers 16: prediction 7min

batch_size = 32, num_workers 20: prediction 5min

batch_size = 32, num_workers 20, pin_memory=True: prediction 4.5min



In [16]:
# Assuming `labels` is the tensor containing predicted labels for each point
predicted_colors = color_map[labels.cpu().numpy()]  # Shape: [num_points, 3]


In [19]:
print(len(predicted_colors))

866900


In [17]:
# Assuming `pcd` is your Open3D point cloud object
downpcd.colors = o3d.utility.Vector3dVector(predicted_colors)

In [18]:
# Visualize the point cloud with colored labels
o3d.visualization.draw_geometries([downpcd])

In [19]:
# Save the point cloud to a file
save_path = "C:/Users/yanpe/OneDrive - Metropolia Ammattikorkeakoulu Oy/Research/data/smartlab/labelled/Smartlab_simulated_transform_label_pointnet2_x3_0.03_20250423.ply"
o3d.io.write_point_cloud(save_path, downpcd)


True

In [21]:
del down_points
del down_colors
torch.cuda.empty_cache()